In [1]:
#Import Libraries
import subprocess
import re
import pandas as pd
import os
import collections
import scipy.stats
import numpy as np
import scipy
import csv
import math
import seaborn as sns
import statsmodels.api as sm

from functools import reduce
from tqdm import tqdm
from matplotlib import pyplot as plt
%matplotlib inline
from pandas.tools.plotting import table
from scipy import interp
from sklearn import svm
from sklearn.svm import SVC
from sklearn.datasets import make_blobs, make_classification


from sklearn.feature_selection import SelectKBest, chi2, VarianceThreshold
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, GridSearchCV, LeaveOneOut, train_test_split, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, auc, roc_curve, precision_recall_curve, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils.multiclass import unique_labels
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D

In [2]:
mirTarBase=pd.read_csv('miRtarbase1.csv', sep='\t', header=None)
TarBase = mirTarBase[0].values
lista=[]

for x in TarBase:
    
    lista.append(x.lower())

In [3]:
T_Base=[]

for x in ["-".join(x.strip().split("-")[:3])  for x in lista]:
    T_Base.append(x) 

In [4]:
giada=set(T_Base)

In [5]:
r_ss=pd.read_csv('new_data/gene_exprs/R_SS_last.csv', sep=',')
r_sl=pd.read_csv('new_data/gene_exprs/R_SL_last.csv', sep=',')
sl_ss=pd.read_csv('new_data/gene_exprs/SS_SL_last.csv', sep=',')
r_stot=pd.read_csv('new_data/gene_exprs/R_Stot_last.csv', sep=',')

In [6]:
dsets=pd.concat([r_ss, r_sl, sl_ss, r_stot])

In [7]:
ent_id=set(dsets['entrez_id'])
type(ent_id)

set

In [8]:
ent_set=set(mirTarBase[1])

In [9]:
intersect=ent_id.intersection(ent_set)

In [22]:
intersect

{38.0,
 780.0,
 2034.0,
 2122.0,
 3280.0,
 3309.0,
 3459.0,
 3791.0,
 5139.0,
 6135.0,
 7855.0,
 7976.0,
 10525.0,
 23492.0,
 51726.0,
 113612.0,
 255488.0}

In [21]:
geni=list(intersect)
geni

[255488.0,
 3459.0,
 23492.0,
 38.0,
 7976.0,
 2122.0,
 780.0,
 113612.0,
 51726.0,
 3309.0,
 3280.0,
 7855.0,
 3791.0,
 5139.0,
 2034.0,
 6135.0,
 10525.0]

In [29]:
ds = dsets[dsets.entrez_id.isin(geni)]
dset_geni=pd.DataFrame(ds)

In [43]:
x=dset_geni[['entrez_id', 'gene_symbol']]
x['gene_symbol'].unique()
#x['entrez_id'].unique()
#x.head()

array(['RPL11', 'EPAS1', 'KDR', 'DDR1', 'HSPA5', 'HYOU1', 'FZD5', 'MECOM',
       'DNAJB11', 'HES1', 'RNF144B', 'FZD3', 'PDE3A', 'CYP2U1', 'IFNGR1',
       'ACAT1', 'CBX7'], dtype=object)

In [11]:
r_ss_=pd.read_csv('new_data/miRNA/SS_R_last.csv', sep=',')
r_sl_=pd.read_csv('new_data/miRNA/SL_R_last.csv', sep=',')
sl_ss_=pd.read_csv('new_data/miRNA/SS_SL_last.csv', sep=',')
r_stot_=pd.read_csv('new_data/miRNA/Stot_R_last.csv', sep=',')

In [12]:
dsets_=pd.concat([r_ss_, r_sl_, sl_ss_, r_stot_])

In [13]:
dsets_.head()

,patient,chrom,start,stop,mirna_id,rpm,entrez_id,gene_symbol
0,SS_00000,chr2,87629754,87629834,hsa-mir-4435-1,0.470089,100616499.0,mir4435-1
1,SS_00000,chr7,44881747,44881800,hsa-mir-4657,0.000000,100616393.0,mir4657
2,SS_00000,chr8,93916021,93916119,hsa-mir-378d-2,0.000000,100616169.0,mir378d2
3,SS_00000,chr9,20502264,20502342,hsa-mir-4474,0.000000,100616441.0,mir4474
4,SS_00000,chr9,134850741,134850807,hsa-mir-3689f,0.000000,100616212.0,mir3689f


In [14]:
mi_id=set(dsets_['mirna_id'])
type(mi_id)

set

In [15]:
intersect_=mi_id.intersection(giada)

In [16]:
intersect_

{'hsa-mir-1180',
 'hsa-mir-181c',
 'hsa-mir-208a',
 'hsa-mir-501',
 'hsa-mir-548j',
 'hsa-mir-656',
 'hsa-mir-760'}